In [56]:
# !py -m pip install opencv-python
# !python.exe -m pip install --upgrade pip
import numpy as np
import cv2 as cv
from numpy import linalg
from math import acos, sqrt, degrees
from matplotlib import pyplot as plt

# odabir levih tacaka

In [57]:
leve8 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("lijeva", slika)
        
        leve8.append([x, y, 1])
        brojac += 1
        
        if brojac == 16:
            cv.imwrite('lijevaOznacena.jpg', slika)
            cv.imshow("lijeva", slika)

In [58]:
putanja = './lijeva.jpg'
slika = cv.imread(putanja)

cv.imshow("lijeva", slika)

cv.setMouseCallback("lijeva", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

In [59]:
np.array(leve8)

array([[ 346,  641,    1],
       [ 184,  760,    1],
       [ 420,  825,    1],
       [ 561,  715,    1],
       [ 367,  864,    1],
       [ 219,  959,    1],
       [ 421, 1046,    1],
       [ 549,  913,    1],
       [ 772,  792,    1],
       [ 695,  841,    1],
       [ 968, 1006,    1],
       [1041,  949,    1],
       [ 771,  827,    1],
       [ 689,  876,    1],
       [ 958, 1043,    1],
       [1031,  985,    1]])

# odabir desnih tacaka

In [60]:
desne8 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("desna", slika)
        
        desne8.append([x, y, 1])
        brojac += 1
        
        if brojac == 16:
            cv.imwrite('desnaOZnacena.jpg', slika)
            cv.imshow("desna", slika)

In [61]:
putanja = './desna.jpg'
slika = cv.imread(putanja)

cv.imshow("desna", slika)

cv.setMouseCallback("desna", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

In [62]:
np.array(desne8)

array([[ 391,  650,    1],
       [ 200,  748,    1],
       [ 352,  819,    1],
       [ 557,  722,    1],
       [ 418,  865,    1],
       [ 253,  937,    1],
       [ 382, 1038,    1],
       [ 568,  926,    1],
       [ 837,  825,    1],
       [ 730,  866,    1],
       [ 948, 1065,    1],
       [1072, 1014,    1],
       [ 835,  859,    1],
       [ 737,  904,    1],
       [ 938, 1115,    1],
       [1065, 1061,    1]])

In [18]:
import numpy
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px

In [20]:
def piksel(koord): #Odredjivanje piksel koordinata tacke
    return np.array([1,-1,-1])*(np.array([1200,0,0])-koord) #promeni 1600 za sliku

In [45]:
# leve8 = piksel(leve8)
leve8

array([[ 856,  642,    1],
       [ 640,  712,    1],
       [ 773,  826,    1],
       [1020,  758,    1],
       [ 231, 1001,    1],
       [ 506,  837,    1],
       [ 429,  794,    1],
       [ 161,  945,    1]])

In [48]:
# desne8 = piksel(desne8)
desne8

array([[ 806,  647,    1],
       [ 644,  718,    1],
       [ 849,  819,    1],
       [1003,  749,    1],
       [ 251, 1066,    1],
       [ 469,  864,    1],
       [ 362,  825,    1],
       [ 131, 1016,    1]])

In [49]:
def fundJed(xl,xd): #Odredjivanje jednacina za fundamentalnu matricu
    return np.array([xl[0]*xd[0],xl[1]*xd[0],xl[2]*xd[0],xl[0]*xd[1],xl[1]*xd[1],xl[2]*xd[1],xl[0]*xd[2],xl[1]*xd[2],xl[2]*xd[2]])

In [50]:
def fundamentalnaMatrica(leve,desne): #Odredjivanje fundamentalne matrice f
    a=[]
    for x,xp in zip(leve,desne):
        jed= fundJed(x,xp)
        a.append(jed)
    a = np.array(a)
    U, D, V = np.linalg.svd(a)
    f = V[-1]
    f = f.reshape(3,3)
    return f

In [51]:
def skew2v(EC): #Uzima vektor iz kososimetricne matrice 
    return np.array([EC[2][1],EC[0][2],EC[1][0]])

In [52]:
def dekomponuj(e): #Dekompozicija osnove matrice E
    q0=np.array([[0,-1,0],[1,0,0],[0,0,1]])
    e0=np.array([[0,1,0],[-1,0,0],[0,0,0]])
    U,S,V=np.linalg.svd(e)
    EC=U@e0@np.transpose(U)
    A=U@q0@V #probaj verziju bez transponovanih q0,v ako nesto ne valja
    return A,skew2v(EC) #skew2v mozda vrati suprotan znak, stavi mu minus ako ne valja

In [53]:
def kamera1(e): #Matrica kamere u koordinatnom sistemu druge kamere
    A,C=dekomponuj(e)
    C=K1 @ -np.transpose(A) @ C
    C=-C
    A=K1 @ np.transpose(A)
    A1=np.concatenate((np.transpose(A),[C]),axis=0) #append A.T,C
    return np.transpose(A1)

In [54]:
def triangulisi(T1,T2,pt1,pt2): #Triangulacija tacaka sa fotografije
    a=np.array([pt1[1]*T1[2,:]-pt1[2]*T1[1,:],
                -pt1[0]*T1[2,:]+pt1[2]*T1[0,:],
                pt2[1]*T2[2,:]-pt2[2]*T2[1,:],
                -pt2[0]*T2[2,:]+pt2[2]*T2[0,:]]) #ima na slajdovima
    U,S,V=np.linalg.svd(a)
    pt=V[3] # valjda moze -1
    pt=(1/pt[3])*pt
    pt=np.where(pt==0,0.0,pt)
    return np.array(pt)

In [27]:
def recPts(c1,c2,pts1,pts2):
    a=[]
    for x,xp in zip(pts1,pts2):
        X=triangulisi(c1,c2,x,xp)
        X=np.delete(X,-1)
        a.append(X)
    a = np.array(a)
    return a

In [55]:
K1=np.array(
        [[1300,0,600],
         [0,1300,800],
         [0,0,1]]
        )
kam2=np.array(
        [[1300,0,600,0],
         [0,1300,800,0],
         [0,0,1,0]]
        )

In [80]:
f=fundamentalnaMatrica(piksel(leve8),piksel(desne8))
#osnovna matrica
e=np.transpose(K1)@f@K1
pts3D=recPts(kamera1(e),kam2,piksel(leve8),piksel(desne8))
print(pts3D)

[[ 0.08577181 -0.08516866  0.58148737]
 [ 0.18488646 -0.04451522  0.61706391]
 [ 0.09874868 -0.00566769  0.53126168]
 [ 0.011839   -0.04327768  0.5102469 ]
 [ 0.07870038  0.01696368  0.57439482]
 [ 0.1618117   0.0518124   0.60843877]
 [ 0.09109701  0.09438552  0.54080929]
 [ 0.0123696   0.04354638  0.52096214]
 [-0.08931457  0.00428718  0.48052154]
 [-0.04990335  0.01909111  0.48789115]
 [-0.10843231  0.08697265  0.39164798]
 [-0.1391823   0.06716594  0.37559151]
 [-0.087933    0.01787643  0.48097993]
 [-0.05289083  0.03455647  0.4960711 ]
 [-0.10872022  0.10367168  0.40370674]
 [-0.14574617  0.08406817  0.39916478]]


In [65]:
print(f)

[[ 3.19388009e-07 -2.76852471e-06  2.36423597e-03]
 [ 3.44161674e-06  7.79747224e-07 -9.26987475e-04]
 [-3.24848655e-03 -8.62580836e-04  9.99991127e-01]]


In [66]:
print(e)

[[ 0.53976574 -4.67880676  0.44336371]
 [ 5.81633229  1.31777281  2.29031445]
 [-0.39462845 -2.46986725 -0.02511179]]


In [71]:
A, E0 = dekomponuj(e)
print(A)

[[-0.65206565 -0.19863484 -0.7316793 ]
 [ 0.08371135 -0.97803134  0.19091124]
 [-0.75352691  0.0632368   0.65436863]]


In [79]:
print("E0")
print(E0)

E0
[ 0.44224085 -0.10066683 -0.89122905]


In [78]:
print("T1:")
print(kamera1(e))

T1:
[[-1.28669292e+03  2.23371501e+02 -5.86963802e+02 -6.83950737e+01]
 [-8.43568727e+02 -1.11871176e+03  6.05702741e+02 -8.00263264e+02]
 [-7.31679296e-01  1.90911239e-01  6.54368632e-01 -9.25989236e-01]]


In [77]:
print("T2:")
print(kam2)

T2:
[[1300    0  600    0]
 [   0 1300  800    0]
 [   0    0    1    0]]


In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# U ovom slucaju crtamo jednostavnu kocku. Vi cete koristiti rekonstruisana temena
temenaKocke = pts3D #np.array([[0,0, 3], [0, 3,3], [3,3,3],[3, 0,3],[0,0,0], [0, 3,0], [3, 3,0],[3,0,0]] )
# Ovo su ivice kocke zadate indeksima temena
ivice = [[0, 1], [0, 3], [0, 4], [2, 3], [2, 1], [2, 6], [5, 6], [5, 4], [5, 1], [7, 6],
[7, 3], [7, 4], [9, 10], [9, 12], [9, 14], [11, 12], [11, 10], [11, 13], [13, 14],
[10, 13], [14, 12], [15, 16], [15, 17], [15, 18], [16, 17], [16, 18], [17, 18]]

# dodatne mogucnosti istrazite sami

#  prikaz scene - koristi "plotly" biblioteku
def prikazKocke(): 
    # izdvajamo x,y,z koordinate svih tacaka
    xdata = (np.transpose(temenaKocke))[0]
    ydata = (np.transpose(temenaKocke))[1]
    zdata = (np.transpose(temenaKocke))[2]
    # u data1 ubacujemo sve sto treba naccrtati
    data1 = []
    # za svaku ivicu crtamo duz na osnovu koordinata
    for i in range(len(ivice)):
        data1.append(go.Scatter3d(x=[xdata[ivice[i][0]], xdata[ivice[i][1]]], y=[ydata[ivice[i][0]], ydata[ivice[i][1]]],z=[zdata[ivice[i][0]], zdata[ivice[i][1]]]))
    fig = go.Figure(data = data1 )
    # da ne prikazuje legendu
    fig.update_layout(showlegend=False)
    # pravi html fajl (ako zelite da napravite "rotatable" 3D rekonstruciju)
    # birate kao parametar velicinu apleta. fulhtml=False je vazno da ne bi pravio ogroman fajl
    # ovde stavite neki vas folder
    fig.write_html("./rekonstrukcija", include_plotlyjs = 'cdn', default_width = '800px', default_height = '600px', full_html = False) #Modifiy the html file
    fig.show()

prikazKocke()

In [ ]:
def centar(A):
    # brisemo kolone redom i nalazimo determinante
    A1 = np.linalg.det(np.delete(A, 0, 1))
    A2 = np.linalg.det(np.delete(A, 1, 1))
    A3 = np.linalg.det(np.delete(A, 2, 1))
    A4 = np.linalg.det(np.delete(A, 3, 1))

    C = (-1/A4)*np.array([A1, -A2, A3, -A4])
    C = np.where(np.isclose(C, 0) , 0.0 , C)  # izbegavanje -0. u rezultatu
    return C

In [ ]:
def kameraA(T):
    T0 = np.delete(T, 3, 1)
    if linalg.det(T0) < 0:
        T0 = -T0
    Q, R = linalg.qr(linalg.inv(T0)) 

    if R[0][0] < 0:
        R = np.diag([-1, 1, 1])@R
        Q = Q@np.diag([-1, 1, 1])
    if R[1][1] < 0:
        R = np.diag([1, -1, 1])@R
        Q = Q@np.diag([1, -1, 1])
    if R[2][2] < 0:
        R = np.diag([1, 1, -1])@R
        Q = Q@np.diag([1, 1, -1])

    A = np.transpose(Q)
    A = np.where(np.isclose(A, 0) , 0.0 , A)  # izbegavanje -0. u rezultatu
    return A


In [ ]:
def kameraK(T):
    T0 = np.delete(T, 3, 1)
    if linalg.det(T0) < 0:
        T0 = -T0
    Q, R = linalg.qr(linalg.inv(T0)) 

    if R[0][0] < 0:
        R = np.diag([-1, 1, 1])@R
        Q = Q@np.diag([-1, 1, 1])
    if R[1][1] < 0:
        R = np.diag([1, -1, 1])@R
        Q = Q@np.diag([1, -1, 1])
    if R[2][2] < 0:
        R = np.diag([1, 1, -1])@R
        Q = Q@np.diag([1, 1, -1])

    K = linalg.inv(R)
    if K[2][2]:
        K = K/K[2][2]
    K = np.where(np.isclose(K, 0) , 0.0 , K)  # izbegavanje -0. u rezultatu
    return K

In [ ]:
def matricaKamere(pts2D, pts3D):
    A = []
    for i in range(len(pts2D)):
        x1, x2, x3 = pts2D[i]
        xp1, xp2, xp3, xp4 = pts3D[i]
        A.append([0, 0, 0, 0, -x3*xp1, -x3*xp2, -x3*xp3, -x3*xp4, x2*xp1, x2*xp2, x2*xp3, x2*xp4])
        A.append([x3*xp1, x3*xp2, x3*xp3, x3*xp4, 0, 0, 0, 0, -x1*xp1, -x1*xp2, -x1*xp3, -x1*xp4])
    A = np.array(A)

    _, _, V = linalg.svd(A)
    T = np.array(V[-1]).reshape(3, 4)
    T /= T[2][3]
    
    T = np.where(np.isclose(T, 0) , 0.0 , T)
    return T

In [ ]:
T = matricaKamere(org, img)

In [ ]:
org =  np.array([1,-1,-1])*(np.array([1600,0,0])- np.array(ts))

In [ ]:
img = np.array([[0,0,3,1], [0,1,3,1], [0,2,3,1],[0,3,3,1],
                [1,0,3,1], [1,1,3,1], [1,2,3,1],[1,3,3,1],
                [2,0,3,1], [2,1,3,1], [2,2,3,1],[2,3,3,1],
                [3,0,3,1], [3,1,3,1], [3,2,3,1],[3,3,3,1], #gornja strana
                [0,3,2,1], [1,3,2,1], [2,3,2,1],[3,3,2,1],
                [0,3,1,1], [1,3,1,1], [2,3,1,1],[3,3,1,1],
                [0,3,0,1], [1,3,0,1], [2,3,0,1],[3,3,0,1], #desna
                [3,0,2,1],[3,1,2,1],[3,2,2,1], [3,3,2,1],
                [3,0,1,1],[3,1,1,1],[3,2,1,1], [3,3,1,1],
                [3,0,0,1],[3,1,0,1],[3,2,0,1], [3,3,0,1]])

In [ ]:
print('Matrica kamere:')
print(T)
print('\nPozicija centra kamere:')
C = centar(T)
print(C)
print('\nSpoljasnja matrica kamere:')
A = kameraA(T)
print(A)
print('\nMatrica kalibracije kamere:')
K = kameraK(T)
print(K)

In [ ]:
n_voxels = np.zeros((3, 3, 3), dtype=bool)
for i in range(3):
    for j in range(3):
        for k in range(3):
            if (i+j+k)%2 == 1:
                n_voxels[i,j,k] = True
filled = np.ones(n_voxels.shape)
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(filled, facecolors='yellow')

ax.quiver(0,0,0, 4,0,0, color='red', label='x', alpha=0.9)
ax.text(4.2, 0, 0, s="X")
ax.quiver(0,0,0, 0,8,0, color='green', label='y', alpha=0.9)
ax.text(0, 8.2, 0, s="Y")
ax.quiver(0,0,0, 0,0,4, color='blue', label='z', alpha=0.9)
ax.text(0, 0, 4.2, s="Z")

ax.quiver(C[0], C[1], C[2], A[0][0], A[0][1], A[0][2], color='red', label='x', alpha=0.9)
ax.quiver(C[0], C[1], C[2], A[1][0], A[1][1], A[1][2], color='green', label='y', alpha=0.9)
ax.quiver(C[0], C[1], C[2], A[2][0], A[2][1], A[2][2], color='blue', label='z', alpha=0.9)
ax.scatter(C[0], C[1], C[2])
ax.text(C[0]+0.2, C[1]+0.1, C[2]+0.8, s="C")


plt.show()

In [ ]:
def afinize(tacka):
    return list(map(lambda x : x / tacka[2], tacka))[:-1]

In [ ]:
def cross(x, y):
    return np.cross(x, y)

In [ ]:
def tacka_nedogleda(t1, t2, t3, t4):
    return list(cross(cross(t1, t2), cross(t3, t4)))

In [ ]:
def zaokruzi(tacka):
    return list(map(lambda x : round(x), tacka))

In [ ]:

# def homogenize(lista_tacaka):
#     for tacka in lista_tacaka:
#         tacka = tacka.append(1)
#     return lista_tacaka

def osmoteme(lista_tacaka):
    #pretvori u homogene
#     lista_tacaka = homogenize(lista_tacaka)
    #izracunaj_tacku
    t1, t2, t3, t5, t6, t7, t8 = lista_tacaka
    
    xb = tacka_nedogleda(t2, t6, t1, t5)
    yb = tacka_nedogleda(t5, t6, t7, t8)
    return afinize(tacka_nedogleda(t8, xb, t3, yb))

In [ ]:
ts

In [ ]:
osmoteme(ts)

In [ ]:
t1, t2, t3, t5, t6, t7, t8 = ts
    
xb = zaokruzi(tacka_nedogleda(t2, t6, t1, t5))
xb

In [ ]:
yb = zaokruzi(tacka_nedogleda(t5, t6, t7, t8))
yb

In [ ]:
tacka_nedogleda(t8, xb, t3, yb)

In [17]:
def osmoteme(ts):
    return [268, 563]